# summary of bout to smooth path transition

## is there a memory effect after dot movement transitions?

### Summary of 3 experiments 20-23 dpf. did not include one experiment at 26dpf where smooth path was much more attractive than zero size path

In [ ]:
%config InteractiveShellApp.pylab_import_all = False
%matplotlib inline
%pylab inline
%reload_ext autoreload
%autoreload 2

import sys
import os
import fnmatch

import numpy as np
import math
import matplotlib.pyplot as plt
import pandas as pd
from pandas import DataFrame, Series
import seaborn as sns

propsFn='props.csv'
props=pd.read_csv(propsFn, header=None, index_col=0, squeeze=True,delim_whitespace=True).to_dict()

base=props['BaseDir']
expFile=props['allExpFn']

RawDataDir = os.path.join(base,props['RawDataDir'])+'\\'
ProcessingDir = os.path.join(base,props['ProcessingDir'])+'\\Fig3\\'
outputDir = os.path.join(base,props['outputDir'])+'\\'

if not os.path.isdir(ProcessingDir):
    os.makedirs(ProcessingDir)
if not os.path.isdir(outputDir):
    os.makedirs(outputDir)

os.chdir(r'C:\\Users\\johannes\\Dropbox\\python\\zFishBehavior\\dishGroupBehavior')

import functions.matrixUtilities_joh as mu
import matplotlib.pyplot as plt
import models.experiment as xp
import models.experiment_set as es
import functions.paperFigureProps as pfp
pfp.paper()
inToCm=2.54


In [ ]:
info=pd.read_csv(expFile, sep=',')#pd.read_csv(expFile,quotechar='"', sep=',', converters={'bdGroup':ast.literal_eval})
info=info[info.stimulusProtocol=='5d']
info

In [ ]:
# collect meta information and save to new csv file for batch processing

aviPath=[]
posPath=[]
PLPath=[]
expTime = []
    
for index,row in info.iterrows():
    startDir=RawDataDir+row.path+'\\'
    #startDir='D:\\data\\b\\2017\\'+row.path+'\\'
    #if not os.path.isdir(startDir):
    #    startDir='E:\\b\\2017\\'+row.path+'\\'
        
    posPath.append(glob.glob(startDir+'PositionTxt*.txt')[0])
    PLPath.append(glob.glob(startDir+'PL*.txt')[0])
    
    head, tail = os.path.split(posPath[-1])
    currTime=datetime.strptime(tail[-23:-4], '%Y-%m-%dT%H_%M_%S')
    expTime.append(currTime)
    
info['txtPath']=posPath
info['pairList']=PLPath

info['epiDur'] = 6      # duration of individual episodes (default: 5 minutes)
info['episodes'] = -1   # number of episodes to process: -1 to load all episodes (default: -1)
info['inDish'] = 10#np.arange(len(posPath))*120     # time in dish before experiments started (default: 10)
info['arenaDiameter_mm'] = 100 # arena diameter (default: 100 mm)
info['minShift'] = 60 # minimum number of seconds to shift for control IAD
info['episodePLcode'] = 0 # flag if first two characters of episode name encode animal pair matrix (default: 0)
info['recomputeAnimalSize'] = 1 # flag to compute animals size from avi file (takes time, default: 1)
info['SaveNeighborhoodMaps'] = 0 # flag to save neighborhood maps for subsequent analysis (takes time, default: 1)
info['computeLeadership'] = 0 # flag to compute leadership index (takes time, default: 1)
info['ComputeBouts'] = 0 # flag to compute swim bout frequency (takes time, default: 1)
info['set'] = np.arange(len(posPath))   # experiment set: can label groups of experiments (default: 0)
info['ProcessingDir']=ProcessingDir
info['outputDir']=outputDir
info['allowEpisodeSwitch']=1

info['expTime']=expTime

csvFile=os.path.join(ProcessingDir,'Fig3.csv')
info.to_csv(csvFile,encoding='utf-8')
info

In [ ]:
def readExperiment(keepData=False):
    tmp=es.experiment_set(csvFile=csvFile)
    if keepData:
        return tmp
    else:
        return 1

expSet=readExperiment(keepData=True)

In [ ]:
csvPath = []
for f in [mu.splitall(x)[-1][:-4] for x in info.txtPath]:
    csvPath.append(glob.glob(ProcessingDir+f+'*siSummary*.csv')[0])

df=pd.DataFrame()
i=0
for fn in csvPath:
    print(fn)
    tmp=pd.read_csv(fn,index_col=0,sep=',')
    tmp.animalSet=i
    tmp.animalIndex=tmp.animalIndex+((i)*15)
    df=pd.concat([df,tmp])
    i+=1
df['episode']=[x.strip().replace('_','') for x in df['episode']]

print('df shape',df.shape)



In [ ]:
d=df.time
r=datetime(2017,1,1)
t2=[pd.to_datetime(x).replace(day=1,month=1)for x in df.time]
t3=[(x-r)/pd.Timedelta('1 hour') for x in t2]
df['t2']=t2
df['t3']=t3
df.head()

In [ ]:
sns.tsplot(data=df, time="t3",value="si",unit="animalIndex",condition="episode",estimator=np.nanmean,interpolate=False,err_style="ci_bars");
plt.xlim([0,24])

In [ ]:
idx=(df['inDishTime']<60*10) & (df['inDishTime']>120)
episodeNames=df['episode'].unique()
dfDR=df[idx]
tmp=dfDR.groupby(['episode','animalIndex'],sort=True)['si']
xax=np.arange(episodeNames.shape[0])

err=tmp.std().unstack().values.T

fig, axes = plt.subplots(figsize=(10, 7))

xt=xax.copy()
xt[0]=0

tmp=tmp.mean().unstack()
tmp['xax']=xax

axes=tmp.plot(x='xax',kind='line',marker='o',yerr=err,
                                  linestyle=':',ax=axes,xticks=xt)


axes.set_ylabel('attraction index')
plt.xlim([0,xax.max()+xax.max()*0.1])

lines, labels = axes.get_legend_handles_labels()
axes.axhline(0,ls=':',color='k')
axes.set_title('Individual Pair Disc Size Tuning, Night vs. Day, n=8 pairs');


In [ ]:
tmp=dfDR.groupby(['episode','animalIndex'],sort=True)['si']
tmp2=tmp.mean().reset_index()
fig, ax = plt.subplots(figsize=(10,10))

plt.axhline(0)
sns.boxplot(x=tmp2['episode'],y=tmp2['si'])
sns.swarmplot(x=tmp2['episode'],y=tmp2['si'],linewidth=1,edgecolor='gray')
plt.ylim([-.1,.6])

In [ ]:
xx=dfDR.groupby(['episode','animalIndex'],sort=True)['si'].mean().reset_index()
useAnimals=np.where(xx[xx.episode==xx.episode.unique()[0]].si>0.1)[0]
useAnimals

In [ ]:
expAll=np.concatenate([x.pair for x in expSet.experiments])
np.shape(expAll)

In [ ]:
df.animalIndex==44

In [ ]:
rng=np.arange(30*60*60*2,30*60*60*12)
IADall=[]
IADall_s=[]
epiAll=[]


for an in useAnimals:
    idx = df.animalIndex==an
    IADcurrent=np.concatenate([expAll[x].IAD() for x in np.where(idx)[0]])[rng]
    IADall.append(IADcurrent)
    IADcurrent=np.concatenate([expAll[x].spIAD_meanTrace() for x in np.where(idx)[0]])[rng]
    IADall_s.append(IADcurrent)
    currSet=df[df.animalIndex==an].animalSet.values[0]
    print(an,currSet)
    epiAll.append(expSet.experiments[currSet].episodeAll[rng])

IADall=np.array(IADall)
IADall_s=np.array(IADall_s)
epiAll=np.array(epiAll)
np.save(ProcessingDir+'IADall.npy',np.squeeze(np.array(IADall)))
np.save(ProcessingDir+'IADall_s.npy',np.squeeze(np.array(IADall_s)))
np.save(ProcessingDir+'epi.npy',epiAll)

In [ ]:
iadStack=[]
iadStack_s=[]
epiStack=[]
startAll=[]
for an in range(IADall.shape[0]):
    epi=epiAll[an]
    subEpi=np.array([int(x[-1])for x in epi])
    transition=(subEpi[1:]==2)&(subEpi[:-1]==1)
    start=np.where(transition)[0]

    for s in start:
        iadStack.append(IADall[an,s-30*60:s+30*60*3])
        iadStack_s.append(IADall[an,s-30*60:s+30*60*3])
        epiStack.append(epi[s])
        startAll.append(s)
        

iadStack=np.array(iadStack)
iadStack_s=np.array(iadStack_s)
epiStack=np.array(epiStack)
startAll=np.array(startAll)
iadStack.shape

In [ ]:
df=pd.DataFrame(iadStack.T)

In [ ]:
df.columns=epiStack
df=df.stack().reset_index()
df.columns=['frame','epi','iad']
df['animal']=np.tile(np.repeat(np.arange(IADall.shape[0]),start.shape[0]),iadStack.shape[1])

In [ ]:
fc=np.tile(startAll,iadStack.shape[1])
df['fc']=df.frame+fc
df['start']=fc
df.shape

In [ ]:
df.head(5)

In [ ]:
labels=np.array(['B-B','B-S','S-S','B-X'])
df['epiNice']=labels[np.unique(df.epi,return_inverse=True)[1]]
df['trial']=np.unique(df.start,return_inverse=True)[1]
df['anTrial']=df.animal+(df.animal.max()*(df.trial))
df[:500:50]

In [ ]:
df['frameR']=30*np.round(df.frame/30.)
df['time']=df.frameR/(30*60.)
dfx=df.groupby(['animal','epiNice','time']).mean().reset_index()
dfx.shape
dfx['expAn']=[['exp1x'+'%02i'%aa] for aa in dfx.animal] 
ff=ProcessingDir+'dfx.csv'
dfx.to_csv(ff)
print(ff)

In [ ]:
dfAll=dfx
newNames=np.array(['I-I-I','I-C-I','I-X-I','C-C-C'])
nameIdx=np.unique(dfAll.epiNice,return_inverse=True)[1]
dfAll['epiNew']=newNames[nameIdx]
dfAll.head()

In [ ]:
pfp.paper()
inToCm=2.54

fig, ax = plt.subplots(figsize=(7/inToCm,4.5/inToCm))

colOrder=np.array([2,0,1,3])
co=[sns.color_palette(None, 8)[co] for co in colOrder]

sns.tsplot(data=dfAll,time='time',value='iad',unit='animal',condition='epiNew',ax=ax,color=co)
ax.axvline(1.0,ls=':',color='k')
ax.axvline(2.5,ls=':',color='k')
ax.set_ylim([25,45])
ax.set_xticks([0,1,2,3,4])
ax.set_xlabel('time [minutes]')
ax.set_ylabel('inter animal \n distance [mm]')
box = ax.get_position()
ax.set_position([box.x0, box.y0, box.width * 0.8, box.height])

# Put a legend to the right of the current axis

handles, labels = ax.get_legend_handles_labels()
# reverse the order
legOrder=np.array([2,3,0,1])
ax.legend([handles[lo] for lo in legOrder], [labels[lo] for lo in legOrder],
         loc='center left', bbox_to_anchor=(1, 0.5))

#ax.legend()

sns.despine()


In [ ]:
pfp.paper()
inToCm=2.54

fig, ax = plt.subplots(figsize=(20/inToCm,10/inToCm))

colOrder=np.array([2,0,1,3])
co=[sns.color_palette(None, 8)[co] for co in colOrder]

sns.tsplot(data=dfAll,time='time',value='iad',unit='animal',condition='epiNew',ax=ax,color=co)
ax.axvline(1.0,ls=':',color='k')
ax.axvline(2.5,ls=':',color='k')
ax.set_ylim([15,45])
ax.set_xticks([0,1,2,3,4])
ax.set_xlabel('time [minutes]')
ax.set_ylabel('inter animal \n distance [mm]')
box = ax.get_position()
ax.set_position([box.x0, box.y0, box.width * 0.8, box.height])

# Put a legend to the right of the current axis
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
sns.despine()


In [ ]:
dfxp=dfAll.pivot_table(index=['frame'],columns=['animal','epiNew'],values='iad')
col=dfxp.columns
dfxp.shape

In [ ]:
dfxpRI=dfxp.reset_index()
dif=dfxpRI-dfxpRI.loc[60].values
nn=np.nanpercentile(dif,90,axis=0)

In [ ]:
nor=np.tile(nn,dfxpRI.shape[0]).reshape((dfxpRI.shape[0],dfxpRI.shape[1]))
nor.shape

In [ ]:
dfxpn=(dfxpRI-dfxpRI.loc[60].values)/nor

dfNew=pd.DataFrame(dfxpn.values[:,1:])
dfNew.columns=col
dfNew['frame']=dfAll.frame.unique()
dfNew
dfxn=dfNew.stack().stack().reset_index()
dfxn.columns=['time','epiNew','anCont','iad']
dfxn.time=dfxn.time/60.
dfxn.head()

In [ ]:
fig, ax = plt.subplots(figsize=(7/inToCm,4.5/inToCm))

useAn=dfxn[dfxn.epiNew=='I-C-I'].pivot_table(index='time',columns='anCont',values='iad').reset_index().values.min(axis=0)>-1
useAn1=np.where(useAn[1:])[0]#
useAn=dfxn[dfxn.epiNew=='I-X-I'].pivot_table(index='time',columns='anCont',values='iad').reset_index().values.min(axis=0)>-1
useAn2=np.where(useAn[1:])[0]#[1:]
use=(dfxn.epiNew.isin(['I-C-I','I-X-I']).values)&((dfxn.anCont.isin(list(useAn1)).values)&(dfxn.anCont.isin(list(useAn2)).values))

sns.tsplot(data=dfxn[use],time='time',value='iad',unit='anCont',condition='epiNew',ax=ax,legend=False)

ax.axvline(1.0,ls=':',color='k')
ax.axvline(2.5,ls=':',color='k')
ax.axhline(0,color='gray')
ax.set_ylim([-.3,1.2])
ax.set_xlim([0.9,3])
ax.set_xticks([1,2,3])
ax.set_yticks([0,1])
ax.set_xlabel('Time [minutes]')
ax.set_ylabel('Normalized dot animal \n distance (mm)')

box = ax.get_position()
ax.set_position([box.x0, box.y0, box.width * 0.8, box.height])


sns.despine()
figPath=outputDir+'\\3C_PathTransition_IadOverTimeNorm.svg'
plt.savefig(figPath)

In [ ]:
fig, ax = plt.subplots(figsize=(20/inToCm,10/inToCm))

dfxn['ts']=dfxn.time*60
sns.tsplot(data=dfxn[use],time='ts',value='iad',unit='anCont',condition='epiNew',ax=ax)
ax.axvline(60,ls=':',color='k')
ax.set_ylim([-.05,0.3])
ax.set_xlim([58,65])
#ax.set_xticks([1,2,3])
#ax.set_yticks([0,1])
ax.set_xlabel('time [seconds]')
ax.set_ylabel('normalized inter animal \n distance [mm]')

box = ax.get_position()
ax.set_position([box.x0, box.y0, box.width * 0.8, box.height])

# Put a legend to the right of the current axis
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
sns.despine()


In [ ]:
dfxn[use].anCont.unique()

In [ ]:
pfp.paper()
inToCm=2.54

fig, ax = plt.subplots(figsize=(7/inToCm,4.5/inToCm))
colOrder=np.array([2,0,1,3])
co=[sns.color_palette(None, 8)[co] for co in colOrder]

sns.tsplot(data=dfAll[dfAll.animal.isin(dfxn[use].anCont.unique())],
           time='time',value='iad',unit='animal',condition='epiNew',ax=ax,color=co)
ax.axvline(1.0,ls=':',color='k')
ax.axvline(2.5,ls=':',color='k')
ax.set_ylim([22,45])
ax.set_xticks([0,1,2,3,4])
ax.set_xlabel('Time (minutes)')
ax.set_ylabel('Dot animal \n distance (mm)')
box = ax.get_position()
ax.set_position([box.x0, box.y0, box.width * 0.8, box.height])

# Put a legend to the right of the current axis
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
sns.despine()

figPath=outputDir+'\\3B_PathTransition_IadOverTime.svg'
plt.savefig(figPath)

In [ ]:
pfp.paper()
inToCm=2.54

fig, ax = plt.subplots(figsize=(20/inToCm,10/inToCm))

colOrder=np.array([2,0,1,3])
co=[sns.color_palette(None, 8)[co] for co in colOrder]

sns.tsplot(data=dfAll[dfAll.animal.isin(dfxn[use].anCont.unique())],
           time='time',value='iad',unit='animal',condition='epiNew',ax=ax,color=co)

ax.axvline(1.0,ls=':',color='k')
ax.axvline(2.5,ls=':',color='k')
ax.set_ylim([22,45])
ax.set_xticks([0,1,2,3,4])
ax.set_xlabel('time [minutes]')
ax.set_ylabel('inter animal \n distance [mm]')
box = ax.get_position()
ax.set_position([box.x0, box.y0, box.width * 0.8, box.height])

# Put a legend to the right of the current axis
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
sns.despine()



In [ ]:
ii=use&(dfxn.epiNew=='I-C-I').values
bs=dfxn[ii].pivot_table(index='time',columns='anCont',values='iad').reset_index().values[:,1:]
plt.plot(bs,'.-')
plt.ylim([-.2,.8])
plt.xlim([50,180])
plt.axhline(0,ls=':')

In [ ]:
ii=use&(dfxn.epiNew=='I-X-I').values
bx=dfxn[ii].pivot_table(index='time',columns='anCont',values='iad').reset_index().values[:,1:]
plt.plot(bx,'.-')
plt.ylim([-.2,.8])
plt.xlim([50,180])
plt.axhline(0,ls=':')

In [ ]:
def getSlope(X,rng):
    t=np.arange(rng.shape[0])
    tmp=np.zeros((X.shape[1],5))
    for i in range(X.shape[1]):
        tmp[i,:]=stats.linregress(t,X[rng,i]) #s,i,r,p,std

    return tmp


In [ ]:
pfp.paper()
inToCm=2.54

fig, ax = plt.subplots(figsize=(2.5/inToCm,4.5/inToCm))

bst=getSlope(bs,np.arange(60,65))[:,0]
bxt=getSlope(bx,np.arange(60,65))[:,0]
#bst=np.argmax(bs[60:80,:]>0.2,axis=0)
#bxt=np.argmax(bx[60:80,:]>0.2,axis=0)

crossThreshold=np.hstack([bst,bxt])
lab=np.array(['I-C-I','I-X-I'])

dfCT=pd.DataFrame({'ds':crossThreshold})
dfCT['stimulus']=np.repeat(lab,bst.shape)
#sns.boxplot(data=dfCT,x='stimulus',y='ds',ax=ax,linewidth=2)
sns.swarmplot(data=dfCT,x='stimulus',y='ds',ax=ax,linewidth=1,edgecolor='gray',zorder=1)
sns.pointplot(data=dfCT,x='stimulus',y='ds',ax=ax,ci='sd',markers='_',color='k',scale=3,join=False,
              errwidth=1,
              capsize=.3,
             zorder=100)
sns.pointplot(data=dfCT,x='stimulus',y='ds',ax=ax,ci=None,markers='_',color='r',scale=3,join=False)
plt.xlabel('Transition')
sns.despine()
ax.set_ylabel('Separation slope \n (%/second)')
figPath=outputDir+'\\3D_dissociationTime.svg'
plt.savefig(figPath)

from shutil import copy2





for i,row in info.iterrows():
    fn=row.txtPath
    head, tail = os.path.split(fn)

    copyList=[]
    copyList.append(glob.glob(head+'\\ROI*.csv')[0])
    copyList.append(glob.glob(head+'\\PositionTxt*.txt')[0])
    copyList.append(glob.glob(head+'\\PL*.txt')[0])
    copyList.append(glob.glob(head+'\\*anSize.csv')[0])
    
    for f in copyList:
        print(f)
        if f[0]=='E':
            keepSlash=3
        else:
            keepSlash=4
        toDirectory = "e:\\b\\LarschAndBaier2018\\RawData\\" + os.path.join(*splitall(f)[keepSlash:-1])+"\\"
        #toDirectory = "e:\\b\\LarschAndBaier2018\\RawData\\" 
        if not os.path.isdir(toDirectory):
            os.makedirs(toDirectory)
        
        copy2(f, toDirectory)
